In [55]:
import csv
import pandas as pd
import pickle
import soundfile as sf
import librosa
import numpy as np

THRESHOLD = 7.5
SAMPLE_LENGTH = 1000

STRIDE_SIZE = 10.
FRAME_SIZE = 25.
N_MFCC = 16

from pathlib import Path
home = str(Path.home())
SONG_DIR = home + "/Downloads/songdata_90/songdata/"

def get_mfcc_features(song_data, sample_rate, stride_size = STRIDE_SIZE, frame_size = FRAME_SIZE):
    mfccs = librosa.feature.mfcc(song_data, sample_rate, 
                                 n_mfcc=N_MFCC,
                                 hop_length=int(STRIDE_SIZE / 1000. * sample_rate), 
                                 n_fft=int(FRAME_SIZE / 1000. * sample_rate))
    return mfccs

def load_song(song_file):
    data, samplerate = sf.read(song_file)
    data = data[:, 0]
    mfccs = get_mfcc_features(data, samplerate)
    mfccs = np.asarray(mfccs).T
    return mfccs

def peaks_to_windows(song_name, good_peaks, bad_peaks):
    mfccs = load_song(song_name)
    good_samples = []
    bad_samples = []
    for p in good_peaks:
        n = int((p[0] - (FRAME_SIZE/1000.)) / (STRIDE_SIZE/1000.))
        good_samples.append(mfccs[n-(SAMPLE_LENGTH//2):n+(SAMPLE_LENGTH//2), :])
    for p in bad_peaks:
        n = int((p[0] - (FRAME_SIZE/1000.)) / (STRIDE_SIZE/1000.))
        bad_samples.append(mfccs[n-(SAMPLE_LENGTH//2):n+(SAMPLE_LENGTH//2), :])
    return good_samples, bad_samples
        
    
    

In [56]:


results_file = open('resultsFile', 'rb')      
results = pickle.load(results_file) 
list.sort(results, key = lambda x : x[0])

df = pd.read_csv("songs_fixed.csv")

detection_count = 0
drop_count = 0
bad_samples_overall = []
good_samples_overall = []

for i in range(len(results)):
    drops = str(df.iloc[[i]]["Drops"][i]).split(", ")
    peaks = results[i][1]
    good_peaks = set()
    temp_detection_count = 0
    songname = df.iloc[[i]]["Song Name"][i]
    drops_found = []
    for d in drops:
        for p in peaks:
            dval = float(d)
            pval = float(p[0])
            if pval >= dval - THRESHOLD and pval <= dval + THRESHOLD:
                temp_detection_count += 1
                drops_found.append(d)
                good_peaks.add(p)
                break
    if temp_detection_count < len(drops):
        print(songname)
        print(drops_found)
    bad_peaks = set(peaks) - good_peaks
    drop_count += len(drops)
    detection_count += temp_detection_count
    
    good_samples, bad_samples = peaks_to_windows(SONG_DIR + "/" + results[i][0], good_peaks, bad_peaks)
    for b in bad_samples:
        bad_samples_overall.append(b)
        
    for g in good_samples:
        good_samples_overall.append(g)

print(detection_count / drop_count)

Ariana_Grande___Side_To_Side_(TRU_Concept_Remix_ft._Romany)
['170.1']
0.9951690821256038


(206, 1000, 16)
